In [8]:
#Fewshot PromptTemplate -> 모델에게 예제를 주는 것
#소문자로 써줘, 콤마로 구분해줘 이렇게 프롬프트로 알려주는 것보다 예제로 알려주는 것
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_template="""
    Huamn : {question}
    AI : {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix = "Human : What do you know about {country}?",
    input_variables = ["country"],
)
prompt.format(country="Germany")

chain = prompt | chat
chain.invoke({
    "country" : "Germany"
})

AI : 
        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI : \n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro')

In [9]:
#FewShoatChatMessagePromptTemplate

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum\n        Currency: Thai Baht\n        ')

In [11]:
#LengthBasedExampleSelector -> 예제를 선택할 수 있고 설정한 예제에 따라 프롬프트 가능
#유저의 로그인 여부, 유저가 사용하는 언어에따라 예제를 선택할 수 있음

from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)] #예제중 무작위로 하나 선택


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")



'Human: What do you know about Greece?\nAI:\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [15]:
#Serialization and Composition 
# -> 디스크에서 prompt templates 가져오고 // 많은 prompt들을 다 모으는 방법(pipeline)


#intro, example, start 프롬프트를 합치려면 Pipelineprompttemplate를 사용하면 된다.
#Pipelineprompttemplate를 사용하려면 final_prompt를 만들어서 합치려는 프롬프트를 순서대로 묶어주고, prompt만들어서 리스트 안에 튜플로 pormpt를 넣어준뒤 새로운 프롬프트에 넣어줘야 한다

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate
#from langchain.prompts import load_prompt

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

#prompt = load_prompt("./prompt.yaml")

intro = PromptTemplate.from_template("""
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template("""
    This is an example of how you talk:
    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template("""
    Start now!
    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template("""
    {intro}                    
    {example}                         
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)



Arrrrg! Me favorite food be a good ol' plate of rum-soaked grog and salted fish! Yarrrr!

AIMessageChunk(content="Arrrrg! Me favorite food be a good ol' plate of rum-soaked grog and salted fish! Yarrrr!")

In [5]:
# caching -> LLM이 똑같은 질문을 받는다면 캐싱처리를 통해 응답

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

ValueError: Please pass in Redis object.

In [6]:
# openAI 모델을 사용할 때 우리가 지출하는 비용을 아는 방법
# 모델을 어떻게 저장하고 불러오는지?

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.llms.openai import OpenAI

chat = OpenAI(temperature=0.1, max_tokens=450, model="gpt-3.5-turbo-16k")
chat.save("model.json")

# chat = ChatOpenAI(
#     temperature=0.1,
# )


# with get_openai_callback() as usage:
#     a = chat.predict("What is the recipe for soju")
#     b = chat.predict("What is the recipe for bread")
#     print(a, "\n")
#     print(b, "\n")
#     print(usage)

Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of nuruk (fermentation starter)
- 1 tablespoon of yeast

Instructions:
1. Rinse the rice under cold water until the water runs clear.
2. In a large pot, combine the rice and water and bring to a boil. Reduce heat to low and simmer for 20 minutes, or until the rice is cooked.
3. Remove the pot from heat and let it cool to room temperature.
4. In a separate bowl, mix the nuruk and yeast with a little bit of warm water to form a paste.
5. Add the paste to the cooled rice and mix well.
6. Cover the pot with a clean cloth and let it ferment in a warm, dark place for 3-4 days.
7. After fermentation, strain the mixture through a cheesecloth to remove any solids.
8. Transfer the liquid to a clean bottle and store in the refrigerator.
9. Serve chilled and enjoy your homemade soju! 

Ingredients:
- 3 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/4 cups warm water
- 1 tablespoon sugar
- 1 teaspoon salt
- 2 tablespoons olive o